# HW2 - Raccolta dati (estrazione dei topic e preprocessing)

## Installazione delle librerie

In [ ]:
import pandas as pd
import openai
import time

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

## Lettura del dataset (formato .csv)

Leggiamo il dataset dei progetti della Federico II, effettuando un preprocessing su alcune colonne del dataset che vengono eliminate perchè non usate ai fini dell'analisi.

In [ ]:
df = pd.read_csv('./Dataset/Dataset_Projects_Unina.csv', header=0)
df.drop(labels=['Abstract_translated','Keywords','Researchers','City_of_Research_organization','Program'], axis=1, inplace=True)

## Estrazione dei topic dal titolo dei progetti

Tramite le API di OpenAI si estrae un topic sintetico che caratterizza il progetto a partire dal suo titolo.
<br><br>
**NOTA:** Si inserisce tale topic come colonna del dataset dei progetti.

In [ ]:
openai.api_key = "INSERIRE API-KEY"

topic = []
for title in df['Title_translated']:
    prompt = "Given the title of the following research project: '{}' generate the hypothetic topic with a word limit of 20".format(title)

    if len(title) != 0:
        #message = {"role": "user", "content": prompt}
        completion = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0, max_tokens=150)
        topic.append(completion['choices'][0]['text'])
    
    print('Titolo: ' + title + ' Topic: ' + completion['choices'][0]['text'] + ' \n')
    time.sleep(5) # Sleep di 5 secondi per evitare errori di RateLimit imposti da OpenAI

df['topic'] = topic

### Lemmatizing dei topic

Si effettua una fase di **lemmatizing**, in cui avviene il processo di normalizzazione linguistica che consiste nel ridurre una parola alla sua forma base (es. da plurale a singolare).
<br><br>
**NOTA:** Per effettuare tale operazione si usa la funzione _WordNetLemmatizer()_ della libreria **NLTK ("Natural Language Toolkit")**.

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w) for w in words])

df['topic'] = df['topic'].apply(lemmatize_text)

## Salvataggio del dataset preprocessato (formato .csv)

In [ ]:
df.to_csv('./Dataset/Dataset_Projects_Unina_with_Topic.csv', index=False)